In [ ]:
# %% [markdown]
# # Wings R Us Recommendation System
# Competition Submission Notebook

# %% [markdown]
# ## 1. Data Loading
import pandas as pd

# Load raw transaction data
raw_data = pd.read_csv('data/raw_data.csv')
print(f"Loaded {len(raw_data)} raw transactions")
raw_data.head()

# %% [markdown]
# ## 2. Data Cleaning
def clean_data(df):
    # Convert items from string to list
    df['items'] = df['items'].str.split(', ')
    
    # Remove empty orders and orders with just 1 item
    df = df[df['items'].apply(len) > 1]
    
    return df

cleaned_data = clean_data(raw_data)
print(f"{len(cleaned_data)} valid transactions after cleaning")
cleaned_data.head()

# %% [markdown]
# ## 3. Model Training
from src.recommendation_engine import Recommender

# Initialize recommender
recommender = Recommender()

# Train models
recommender.train(cleaned_data)
print("Model training completed!")

# %% [markdown]
# ## 4. Generate Recommendations
def generate_output(test_cases_path, output_path):
    # Load test cases (provided by competition)
    test_cases = pd.read_excel(test_cases_path)
    
    # Prepare recommendations
    results = []
    for _, row in test_cases.iterrows():
        current_items = row['current_items'].split(', ')
        recs = recommender.recommend(current_items, k=3)
        
        results.append({
            'customer_id': row['customer_id'],
            'current_items': row['current_items'],
            'recommendation_1': recs[0],
            'recommendation_2': recs[1],
            'recommendation_3': recs[2]
        })
    
    # Save to Excel
    pd.DataFrame(results).to_excel(output_path, index=False)
    print(f"Recommendations saved to {output_path}")

# Generate output file
generate_output('data/test_cases.xlsx', 'TeamName_Recommendation_Output_Sheet.xlsx')

# %% [markdown]
# ## 5. Validation (Optional)
def validate_recommendations():
    try:
        output = pd.read_excel('TeamName_Recommendation_Output_Sheet.xlsx')
        truth = pd.read_excel('data/ground_truth.xlsx')
        
        merged = pd.merge(output, truth, on='customer_id')
        merged['hit'] = merged.apply(
            lambda x: x['missing_item'] in [x['recommendation_1'], 
                                          x['recommendation_2'],
                                          x['recommendation_3']], axis=1)
        
        accuracy = merged['hit'].mean()
        print(f"Validation Accuracy: {accuracy:.2%}")
    except FileNotFoundError:
        print("Validation files not found - skipping")

validate_recommendations()